# Hyperparameter Tuning using HyperDrive

Importing Dependencies.

In [2]:
from azureml.core import Workspace,Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice,uniform
from azureml.train.sklearn import SKLearn

## Dataset

Getting Dataset

In [3]:
ws = Workspace.from_config()
experiment_name = 'heart_faliure'

experiment = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

path = "https://raw.githubusercontent.com/Sara-cos/Heart_Faliure_PredictionModel/main/starter_file/heart_failure_clinical_records_dataset.csv"

dataset = Dataset.Tabular.from_delimited_files(path)


Workspace name: quick-starts-ws-142919
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-142919


In [16]:
# compute target
cpu_compute_name = "cpucompute"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_compute_name)
    print('Using existing cluster...')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_co_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Using existing cluster...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [24]:
# Creating an early termination policy. (This is not required if we are using Bayesian sampling.)
early_termination_policy = BanditPolicy(evaluation_interval = 2,slack_factor = 0.1)

# Creating different params that will be used during training
param_sampling = RandomParameterSampling({
                "--C": uniform(0.001, 1.0),
                "--max_iter": choice(50, 100, 200)
                })

# Creating your estimator and hyperdrive config
estimator = SKLearn(source_directory='.',entry_script='train.py',compute_target=cpu_cluster)

hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                    hyperparameter_sampling = param_sampling,
                                    policy = early_termination_policy,
                                    primary_metric_name = "Accuracy",
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 4,
                                    max_concurrent_runs = 4)

In [25]:
exp = experiment.submit(config=hyperdrive_run_config, show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [26]:
RunDetails(exp).show()
exp.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_53eff150-6f80-471c-be6f-06ab46640b2f
Web View: https://ml.azure.com/runs/HD_53eff150-6f80-471c-be6f-06ab46640b2f?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-142919/workspaces/quick-starts-ws-142919&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-04-17T09:13:31.801226][API][INFO]Experiment created<END>\n""<START>[2021-04-17T09:13:32.404785][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-04-17T09:13:32.5289297Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-04-17T09:13:32.580512][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_53eff150-6f80-471c-be6f-06ab46640b2f
Web View: https://ml.azure.com/runs/HD_53eff150-6f80-471c-be6f-06ab46640b2f?wsid=/subscriptions/610d6e37-4747-4a20-8

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = exp.get_best_run_by_primary_metric()
print('Best Run Id: ', best_run.id)
best_run_metrics = best_run.get_metrics()
print(best_run_metrics)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name = 'heart-failure-prediction-hyperdrive-model',model_path ='outputs/hd-model.joblib', tags = {'Method':'Hyperdrive'}, properties = {'Accuracy': best_run_metrics['Accuracy']})
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service